# Updatable Matplotlib Plot

The following example shows how you might create a matplotlib plot which can be updated based on user interactions. In this simple case the plot updates each time a user clicks on the graph, however you can imaging more complex scenarios in which users select datasets from drop down menus or interact with sliders.

# A Little Set Up

In [1]:
%matplotlib agg

from example_utils import localhost, pretty_dict_string

webpage_url = localhost('http', 8765) + '/idom/client/index.html'
websocket_url = localhost('ws', 8765) + '/idom/stream'

print('Get a webpage that streams layout updates via a websocket:')
print(webpage_url)
print()
print('Websockets use this route to stream layout updates:')
print(websocket_url)
print()

import idom
import matplotlib.pyplot as plt

swap, element = idom.hotswap()
idom.SimpleServer(element).daemon("0.0.0.0", 8765, access_log=False)

Get a webpage that streams layout updates via a websocket:
http://127.0.0.1:8765/idom/client/index.html

Websockets use this route to stream layout updates:
ws://127.0.0.1:8765/idom/stream



<Thread(Thread-4, started 139724282111744)>

[2019-05-03 13:10:21 -0700] [19683] [INFO] Goin' Fast @ http://0.0.0.0:8765


# The Code

At the moment iDOM doesn't have native support for matplotlib plots, however you can still render an image by:

1. Creating an `Image` element with some format (e.g. SVG).

2. Calling a figure's `savefig` method and writing to the `Image.io` attribute.

In [2]:
from random import random


@idom.element
async def Figure(self):
    x = range(100)
    y = [random() for _ in x]
    img = plot(x, y)

    events = idom.Events()
    
    @events.on("click")
    async def scramble():
        self.update()

    return idom.nodes.div(img, eventHandlers=events)


def plot(*args, **kwargs):
    fig, axes = plt.subplots()
    axes.plot(*args, **kwargs)
    img = idom.Image("svg")
    fig.savefig(img.io, format="svg")
    plt.close(fig)
    return img


swap(Figure)

# Try it Out

In [3]:
idom.display("jupyter", websocket_url)

JupyterWigdet('ws://127.0.0.1:8765/idom/stream')